In [1]:
import os
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torch import autograd
from torch.autograd import Variable
from torchvision.utils import make_grid
import matplotlib.pyplot as plt

In [49]:
def data_read(path):
    readed_data = []
    for image_name in os.listdir(path):
        image_path = os.path.join(path, image_name)
        image = Image.open(image_path)
        image = image.convert("RGB")
        readed_data.append(image)
    return readed_data

class Apple_Dataset(Dataset):
    def __init__(self, path, transform= None) -> None:
        super().__init__()
        self.transform = transform
        self.images = data_read(path)
        print("sizes: ", len(self.images))
    
    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        image = self.images[index]
        if self.transform:
            image = self.transform(image)
        return image
    

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

In [36]:
red_apples_path = r"C:\Python_Projects\esaditional_GAN\conditional-GAN\datasett\red_apples"
green_apples_path = r"C:\Python_Projects\esaditional_GAN\conditional-GAN\datasett\green_apples"

batch_size = 8
epochs = 5

input_size = 4
encode_channel_sizes = [16, 32, 64]
encode_kernel_sizes = [3, 3, 3]

decode_channel_sizes = [64, 32, 16]
decode_kernel_sizes = [3, 3, 3]

d_input_size = 6
d_channel_sizes = [16, 32, 64]
d_kernel_sizes = [3, 3, 3]
d_ann_sizes = []



In [50]:
red_dataset = Apple_Dataset(red_apples_path, transform=transform)
red_data_loader = torch.utils.data.DataLoader(red_dataset, batch_size, shuffle= True)

sizes:  67


In [7]:
green_dataset = Apple_Dataset(green_apples_path, transform = transform)
green_data_loader = torch.utils.data.DataLoader(green_dataset, batch_size, shuffle= True)

sizes:  67


In [52]:
red_dataset[0].size()

torch.Size([3, 256, 256])

### Generator and Discriminator blocks

In [103]:
class Generator(nn.Module):
    def __init__(self, 
                input_channels: int,
                encode_channel_sizes: list,
                encode_kernel_sizes: list, 

                decode_channel_sizes: list, 
                decode_kernel_sizes: list,
                 
                dropout: float, 
                ) -> None:
        
        self.input_channels: int = input_channels
        self.encode_channel_sizes: list = encode_channel_sizes
        self.encode_kernel_sizes: list = encode_kernel_sizes

        self.decode_channel_sizes: list = decode_channel_sizes
        self.decode_kernel_sizes: list = decode_kernel_sizes

        self.dropout: float = dropout

        super().__init__()
        
        self.model = nn.Sequential(
            nn.Conv2d(self.input_channels, self.encode_channel_sizes[0], self.encode_kernel_sizes[0], padding= "same"), # 4-256-256 -> 16-256-256
            #nn.Conv2d(self.encode_channel_sizes[0], self.encode_channel_sizes[0], self.encode_channel_sizes[0], padding= "same"), # 16-256-256 -> 16-256-256
            nn.ReLU(),
            nn.MaxPool2d(kernel_size= 2, stride= 2), # 16-256-256 -> 16-128-128
            nn.Dropout(self.dropout),

            nn.Conv2d(self.encode_channel_sizes[0], self.encode_channel_sizes[1], self.encode_kernel_sizes[1], padding= "same"), #16-128-128 -> 32-128-128
            #nn.Conv2d(self.encode_channel_sizes[1], self.encode_channel_sizes[1], self.encode_kernel_sizes[1], padding= "same"), #32-128-128 -> 32-128-128
            nn.ReLU(),
            nn.MaxPool2d(kernel_size= 2, stride= 2), #32-128-128 -> 32-64-64
            nn.Dropout(self.dropout),    

            nn.Conv2d(self.encode_channel_sizes[1], self.encode_channel_sizes[2], self.encode_kernel_sizes[2], padding= "same"), # 32-64-64 -> 64-64-64
            #nn.Conv2d(self.encode_channel_sizes[2], self.encode_channel_sizes[2], self.encode_kernel_sizes[2], padding= "same"), # 64-64-64 -> 64-64-64
            nn.ReLU(),
            nn.MaxPool2d(kernel_size= 2, stride= 2), # 64-64-64 -> 64-32-32
            nn.Dropout(self.dropout),

            #here may come some ANN layers 
            #idk why :D
            
            nn.ConvTranspose2d(self.encode_channel_sizes[2], self.decode_channel_sizes[0], self.decode_kernel_sizes[0], stride= 2), # 64-64-64 -> 32-128-128
            nn.Conv2d(self.decode_channel_sizes[0], self.decode_channel_sizes[0], self.decode_kernel_sizes[0], padding= "same"), # 32-128-128 -> 32-128-128
            #nn.Conv2d(self.decode_channel_sizes[0], self.decode_channel_sizes[0], self.decode_kernel_sizes[0], padding= "same"), # 32-128-128 -> 32-128-128
            nn.ReLU(),
            nn.Dropout(self.dropout),

            nn.ConvTranspose2d(self.decode_channel_sizes[0], self.decode_channel_sizes[1], self.decode_kernel_sizes[1], stride= 2), # 32-128-128 -> 16-256-256 
            nn.Conv2d(self.decode_channel_sizes[1], self.decode_channel_sizes[1], self.decode_kernel_sizes[1], padding= "same"), # 16-256-256 -> 16-256-256
            #nn.Conv2d(self.decode_channel_sizes[1], self.decode_channel_sizes[1], self.decode_kernel_sizes[1], padding= "same"), # 16-256-256 -> 16-256-256
            nn.ReLU(),
            nn.Dropout(self.dropout),

            nn.ConvTranspose2d(self.decode_channel_sizes[1], self.decode_channel_sizes[2], self.decode_kernel_sizes[2]), # 16-256-256 -> 3-256-256
            nn.Conv2d(self.decode_channel_sizes[2], self.decode_channel_sizes[2], self.decode_kernel_sizes[2], padding= "same"), # 3-256-256 -> 3-256-256
            #nn.Conv2d(self.decode_channel_sizes[2], self.decode_channel_sizes[2], self.decode_kernel_sizes[2], padding= "same"), # 3-256-256 -> 3-256-256
            nn.ReLU(),
            nn.Dropout(self.dropout)
            )
        
    def forward(self, noise: torch.Tensor, lable_image: torch.Tensor):
        
        x = torch.cat([noise, lable_image], 2)

        out = self.model(x)

        return out

In [99]:
class Discriminator(nn.Module):
    def __init__(self,
                 input_chanel,
                 channel_sizes,
                 kernel_sizes,
                 
                 ann_layer_sizes,
                 ) -> None:
        
        self.input_channel: int = input_chanel
        self.channel_sizes: list = channel_sizes
        self.kernel_sizes: list  = kernel_sizes

        self.ann_layer_sizes: list = ann_layer_sizes

        super().__init__()
        
        self.model = nn.Sequential(
            nn.Conv2d(self.input_channel, self.channel_sizes[0], self.kernel_sizes[0], padding= "same"), # 6-256-256 -> 16-256-256
            #nn.Conv2d(self.channel_sizes[0], self.channel_sizes[0], self.kernel_sizes[0], padding= "same"), # 16-256-256 -> 16-256-256
            nn.ReLU(),
            nn.MaxPool2d(kernel_size= 2, stride= 2), # 16-256-256 -> 16-128-128

            nn.Conv2d(self.channel_sizes[0], self.channel_sizes[1], self.kernel_sizes[1], padding= "same"), # 16-128-128 -> 32-128-128
            #nn.Conv2d(self.channel_sizes[1], self.channel_sizes[1], self.kernel_sizes[1], padding= "same"), # 32-128-128 -> 32-128-128
            nn.ReLU(),
            nn.MaxPool2d(kernel_size= 2, stride= 2), # 32-128-128 - 32-64-64

            nn.Conv2d(self.channel_sizes[1], self.channel_sizes[2], self.kernel_sizes[2], padding= "same"), # 32-64-64 -> 64-64-64
            #nn.Conv2d(self.channel_sizes[2], self.channel_sizes[2], self.kernel_sizes[2], padding= "same"), # 64-64-64 -> 64-64-64
            nn.ReLU(),
            nn.MaxPool2d(kernel_size= 2, stride= 2), # 64-64-64 -> 64-32-32
            
            nn.Flatten(),

            nn.Linear(self.ann_layer_sizes[0], self.ann_layer_sizes[1]),
            nn.ReLU(),
            nn.Linear(self.ann_layer_sizes[1], self.ann_layer_sizes[2]),    
            nn.ReLU(),
            nn.Linear(self.ann_layer_sizes[2], self.ann_layer_sizes[3]),
            nn.ReLU(),
            nn.Linear(self.ann_layer_sizes[3], 1),
            nn.Sigmoid()

        )
    
    def forward(self, real_image, generated_image):
        x = torch.cat([real_image, generated_image], 2)
        out = self.model(x)
        return out

    #its gonna be sequnce of conv2d layers an dense layersc  
    #conv2d will demolishing the image then dense layers with 1 output layer proces the image
    #probably :D

### Train Function

In [100]:
# Loss Function

criterion = nn.BCELoss()

In [102]:
#Optimizer

generator_optimizer = torch.optim.Adam(Generator.parameters)
discriminator_optimizer = torch.optim.Adam(Discriminator.parameters)

TypeError: 'function' object is not iterable

In [95]:
def generator_train_step(batch_size: int, instance_shape: list, discriminator, generator, generator_optimizer, criterion, label_image):

    generator_optimizer.zero_grad()

    noise = torch.randn(batch_size, instance_shape[0], instance_shape[1])
    
    altered_image = generator(noise, label_image)

    validity = discriminator(altered_image, label_image)

    g_loss = criterion(validity, torch.ones(batch_size))
    
    g_loss.backward()

    generator_optimizer.step()

    return g_loss.data

In [96]:
def discriminator_train_step(batch_size, discriminator, generator, discriminator_optimizer, criterioni, real_image, other_real_image, instance_shape):
    
    discriminator_optimizer.zero_grad()

    real_validity = (real_image, other_real_image)

    real_loss = criterion(real_validity, torch.ones(batch_size))

    noise = torch.randn(batch_size, instance_shape[0], instance_shape[1])

    altered_images = generator(noise, other_real_image)

    altered_image_validity = discriminator(altered_images, other_real_image)

    altered_image_loss = criterion(altered_image_validity, torch.ones(batch_size))

    discriminator_loss = altered_image_loss, real_loss

    discriminator_loss.backward()

    discriminator_loss.step()

    return discriminator_loss.data

In [ ]:
for epoch in range(epochs):

    print("Starting epoch {}...".format(epoch + 1))

    
